In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from scipy.ndimage import filters, zoom
from scipy.signal import savgol_filter
from tensorflow import keras

In [ ]:
def generate_occlusion_map(size):
    return np.random.choice(a=[False, True], size=(size, size), p=[0.8, 0.2])

plt.imshow(generate_occlusion_map(32))

In [ ]:
def generate_lights(occlusion_map, num_lights = 10):
    lights = []
    while num_lights:
        row, col = np.random.randint(low=0, high=occlusion_map.shape[0], size=2)
        if occlusion_map[row, col]:
            continue
        lights.append((row, col))
        num_lights -= 1
    return lights

In [ ]:
def ray_occlusion_test(occlusion_map, start, end):
    sx, sy = start
    ex, ey = end
    steep = abs(ey - sy) > abs(ex - sx)
    if steep:
        sx, sy = sy, sx
        ex, ey = ey, ex
    if sx > ex:
        sx, ex = ex, sx
        sy, ey = ey, sy
    dx = ex - sx
    dy = ey - sy
    df = dy / dx
    if dx == 0.0:
        return True
    for x in [sx, ex] + [ix + 0.5 for ix in range(int(sx + 1), int(ex))]:
        ix = int(x)
        iy = int(sy + df * (x - sx))
        if ix >= occlusion_map.shape[0] or iy >= occlusion_map.shape[1]:
            continue
        if not steep and occlusion_map[iy, ix]:
            return False
        elif steep and occlusion_map[ix, iy]:
            return False
        if ix > sx:
            iy = int(sy + df * (ix - sx))
            if not steep and occlusion_map[iy, ix]:
                return False
            elif steep and occlusion_map[ix, iy]:
                return False
        if ix + 1 < ex:
            iy = int(sy + df * (ix + 1 - sx))
            if not steep and occlusion_map[iy, ix]:
                return False
            elif steep and occlusion_map[ix, iy]:
                return False
    return True

In [ ]:
def generic_combined_map(occlusion_map, lights, light_map):
    size = occlusion_map.shape[0]
    ret = np.zeros(shape=(RASTER_SIZE * size, RASTER_SIZE * size, 3), dtype=np.uint8)

    for row in range(ret.shape[0]):
        for col in range(ret.shape[1]):
            x = col // RASTER_SIZE
            y = row // RASTER_SIZE
            s = (0.5 + (col % RASTER_SIZE)) / RASTER_SIZE
            t = (0.5 + (row % RASTER_SIZE)) / RASTER_SIZE
            if occlusion_map[y, x]:
                ret[row, col, 0:3] = [255, 0, 0]
            else:
                ret[row, col, 0:3] = (light_map[row, col] * 255).astype(np.uint8)

    for light in lights:
        ly, lx = RASTER_SIZE * light[0], RASTER_SIZE * light[1]
        for row in range(ly + 2, ly + RASTER_SIZE - 2):
            for col in range(lx + 2, lx + RASTER_SIZE - 2):
                ret[row, col] = [0, 0, 255]
                
    return ret                

In [ ]:
%%time

RASTER_SIZE = 8
ATTENUATION = 1
TRUNCATION = 1 / 255.0

def generate_light_map(occlusion_map, lights):
    ret = np.zeros(shape=RASTER_SIZE * np.array(occlusion_map.shape), dtype=np.float)

    # Precompute labels.
    for row in range(ret.shape[0]):
        for col in range(ret.shape[1]):
            x = col // RASTER_SIZE
            y = row // RASTER_SIZE
            s = (0.5 + (col % RASTER_SIZE)) / RASTER_SIZE
            t = (0.5 + (row % RASTER_SIZE)) / RASTER_SIZE
            for light in lights:
                lx, ly = light[1] + 0.5, light[0] + 0.5
                if not ray_occlusion_test(occlusion_map, (x + s, y + t), (lx, ly)):
                    continue
                irradiance = ATTENUATION / ((x + s - lx)**2 + (y + t - ly)**2)
                if irradiance >= TRUNCATION:
                    ret[row, col] = max(ret[row, col], min(1, ret[row, col] + irradiance))
    
    return ret

In [ ]:
%%time

# Generate our training scene and test scene.
train_occlusion_map = generate_occlusion_map(32)
train_lights = generate_lights(train_occlusion_map, num_lights=12)
train_light_map_raw = generate_light_map(train_occlusion_map, train_lights)
train_light_map = filters.gaussian_filter(train_light_map_raw, sigma = 2.0)

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(generic_combined_map(train_occlusion_map, train_lights, train_light_map))

In [ ]:
%%time

# Generate our training scene and test scene.
train_weights = np.absolute(train_light_map_raw - train_light_map)**0.5
train_weights = np.clip(0.1 + 4.0 * filters.gaussian_filter(train_weights, sigma = 2.0), 0, 1)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(generic_combined_map(train_occlusion_map, train_lights, train_weights))

In [ ]:
# Generate our training scene and test scene.
test_occlusion_map = generate_occlusion_map(32)
test_lights = generate_lights(test_occlusion_map, num_lights=12)
test_light_map = filters.gaussian_filter(generate_light_map(test_occlusion_map, test_lights), sigma = 2.0)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(generic_combined_map(test_occlusion_map, test_lights, test_light_map))

In [ ]:
LIGHT_RAY_SIZE = 9
LIGHTS_PER_VERTEX = 4

def compute_light_ray(occlusion_map, x, y, lx, ly, dx, dy):
    distance = np.linalg.norm([lx - x, ly - y, 0])
    assert distance > 0.01
    return [
        1,
        1.0 / distance**2,
        dx,
        dy,
        occlusion_map[y - 1, x - 1] if y > 0 and x > 0 else 1,
        occlusion_map[y - 1, x] if y > 0 and x < occlusion_map.shape[1] - 1 else 1,
        occlusion_map[y, x - 1] if y < occlusion_map.shape[0] - 1 and x > 0 else 1,
        occlusion_map[y, x] if y < occlusion_map.shape[0] - 1 and x < occlusion_map.shape[1] - 1 else 1,
        int(ray_occlusion_test(occlusion_map, (x, y), (lx, ly))),
    ]

def aggregate_light_ray(occlusion_map, lights, x, y, dx, dy):
    ray = np.zeros(shape=LIGHTS_PER_VERTEX * LIGHT_RAY_SIZE, dtype=np.float32)
    face_lights = sorted(lights, key=lambda l: (l[1] - x)**2 + (l[0] - y)**2)
    for i, light in enumerate(face_lights[:LIGHTS_PER_VERTEX]):
        ly, lx = light[0] + 0.5, light[1] + 0.5
        ray[i * LIGHT_RAY_SIZE : (i + 1) * LIGHT_RAY_SIZE] = compute_light_ray(occlusion_map, x, y, lx, ly, dx, dy)
    return ray

def generate_examples(occlusion_map, lights, light_map, weight_map=None):
    if not weight_map:
        weight_map = np.ones(shape=light_map.shape, dtype=np.float32)

    # Generate training examples.
    labels = []
    examples = [[], [], [], []]
    weights = []
    for y in range(occlusion_map.shape[0]):
        for x in range(occlusion_map.shape[1]):
            # Define the label
            col_s, row_s = x * RASTER_SIZE, y * RASTER_SIZE
            col_e, row_e = (x + 1) * RASTER_SIZE, (y + 1) * RASTER_SIZE
            labels.append(light_map[row_s : row_e, col_s : col_e].flatten())
            
            # Define the example pixel weights
            weights.append(weight_map[row_s : row_e, col_s : col_e].flatten())

            # Generate features by aggregating light rays at each face vertex.
            light_00 = aggregate_light_ray(occlusion_map, lights, x, y, 1, 1)
            light_10 = aggregate_light_ray(occlusion_map, lights, x + 1, y, -1, 1)
            light_01 = aggregate_light_ray(occlusion_map, lights, x, y + 1, 1, -1)
            light_11 = aggregate_light_ray(occlusion_map, lights, x + 1, y + 1, -1, -1)
            
            examples[0].append(light_00)
            examples[1].append(light_10)
            examples[2].append(light_01)
            examples[3].append(light_11)
            
    return examples, np.array(labels), np.array(weights)

In [ ]:
foo_map = np.zeros(shape=train_light_map.shape, dtype=np.float32)

for y in range(train_occlusion_map.shape[0]):
    for x in range(train_occlusion_map.shape[1]):
        light_00 = aggregate_light_ray(train_occlusion_map, train_lights, x, y, 1, 1)
        light_10 = aggregate_light_ray(train_occlusion_map, train_lights, x + 1, y, -1, 1)
        light_01 = aggregate_light_ray(train_occlusion_map, train_lights, x, y + 1, 1, -1)
        light_11 = aggregate_light_ray(train_occlusion_map, train_lights, x + 1, y + 1, -1, -1)

        col_s, row_s = x * RASTER_SIZE, y * RASTER_SIZE
        col_e, row_e = (x + 1) * RASTER_SIZE, (y + 1) * RASTER_SIZE
        for i_s in range(RASTER_SIZE):
            for i_t in range(RASTER_SIZE):
                col = x * RASTER_SIZE + i_s
                row = y * RASTER_SIZE + i_t
                s = (i_s + 0.5) / RASTER_SIZE
                t = (i_t + 0.5) / RASTER_SIZE
                
                for i in range(LIGHTS_PER_VERTEX):
                    if light_00[i * LIGHT_RAY_SIZE + 8]:
                        foo_map[row, col] += (1 - s) * (1 - t) * light_00[i * LIGHT_RAY_SIZE + 1]
                    if light_10[i * LIGHT_RAY_SIZE + 8]:
                        foo_map[row, col] += s * (1 - t) * light_10[i * LIGHT_RAY_SIZE + 1]
                    if light_01[i * LIGHT_RAY_SIZE + 8]:
                        foo_map[row, col] += (1 - s) * t * light_01[i * LIGHT_RAY_SIZE + 1]
                    if light_11[i * LIGHT_RAY_SIZE + 8]:
                        foo_map[row, col] += s * t * light_11[i * LIGHT_RAY_SIZE + 1]
                    foo_map[row, col] = np.clip(foo_map[row, col], 0, 1)
                    
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(generic_combined_map(train_occlusion_map, train_lights, foo_map))

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(generic_combined_map(train_occlusion_map, train_lights, train_light_map))

In [ ]:
%%time

examples, labels, weights = generate_examples(train_occlusion_map, train_lights, train_light_map)

In [ ]:
model = keras.Sequential()

# Define 00-vertex model.
input_00 = keras.layers.Input(shape=(LIGHTS_PER_VERTEX * LIGHT_RAY_SIZE,))
normalize_00 = keras.layers.BatchNormalization()(input_00)
hidden_00_1 = keras.layers.Dense(16, activation="relu")(normalize_00)

# Define 10-vertex model.
input_10 = keras.layers.Input(shape=(LIGHTS_PER_VERTEX * LIGHT_RAY_SIZE,))
normalize_10 = keras.layers.BatchNormalization()(input_10)
hidden_10_1 = keras.layers.Dense(16, activation="relu")(normalize_10)

# Define 01-vertex model.
input_01 = keras.layers.Input(shape=(LIGHTS_PER_VERTEX * LIGHT_RAY_SIZE,))
normalize_01 = keras.layers.BatchNormalization()(input_01)
hidden_01_1 = keras.layers.Dense(16, activation="relu")(normalize_01)

# Define 11-vertex model.
input_11 = keras.layers.Input(shape=(LIGHTS_PER_VERTEX * LIGHT_RAY_SIZE,))
normalize_11 = keras.layers.BatchNormalization()(input_11)
hidden_11_1 = keras.layers.Dense(16, activation="relu")(normalize_11)

# Merge all of the lights together.
combined_input = keras.layers.Add()([
    hidden_00_1,
    hidden_10_1,
    hidden_01_1,
    hidden_11_1,
])
combined_hidden_1 = keras.layers.Dense(16, activation="relu")(combined_input)
combined_output = keras.layers.Dense(RASTER_SIZE * RASTER_SIZE, activation="sigmoid")(combined_hidden_1)

# Input layer.
model = keras.models.Model(
    inputs=[input_00, input_10, input_01, input_11],
    outputs=combined_output,
)

model.compile(
    optimizer=tf.train.AdamOptimizer(
        learning_rate=0.001,
    ),
    loss='mse',    
)

loss = []

In [ ]:
%%time

history = model.fit(
    examples,
    labels,
    class_weight=weights,
    epochs=1000,
    batch_size=1000,
)

In [ ]:
loss = loss + history.history["loss"]
plt.plot(loss[-2000:])

In [ ]:
def plot_predictions(predictions, occlusion_map, lights):
    # Plot the current lighting of the trained model.
    model_map = np.zeros(shape=RASTER_SIZE * np.array(occlusion_map.shape), dtype=np.float32)
    for y in range(occlusion_map.shape[0]):
        for x in range(occlusion_map.shape[1]):
            row_s, row_e = y * RASTER_SIZE, (y + 1) * RASTER_SIZE
            col_s, col_e = x * RASTER_SIZE, (x + 1) * RASTER_SIZE
            P = predictions[x + occlusion_map.shape[1] * y]
            model_map[row_s : row_e, col_s : col_e] = P.reshape(RASTER_SIZE, RASTER_SIZE)

    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(generic_combined_map(occlusion_map, lights, model_map))

In [ ]:
plot_predictions(model.predict(examples), train_occlusion_map, train_lights)

In [ ]:
test_examples, test_labels, _ = generate_examples(test_occlusion_map, test_lights, test_light_map)
plot_predictions(model.predict(test_examples), test_occlusion_map, test_lights)

history = model.evaluate(test_examples, test_labels)
print(history)

history = model.evaluate(examples, labels)
print(history)

In [ ]:
# Regenerate training examples.
train_occlusion_map = generate_occlusion_map(128)
train_lights = generate_lights(train_occlusion_map, num_lights=240)
train_light_map = filters.gaussian_filter(generate_light_map(train_occlusion_map, train_lights), sigma = 2.0)
examples, labels = generate_examples(train_occlusion_map, train_lights, train_light_map) 

In [ ]:
from IPython.display import SVG
from tensorflow.python.keras.utils import vis_utils

SVG(vis_utils.model_to_dot(model).create(prog='dot', format='svg'))